# Writing data to and reading data from a Database using Python

## Libraries and settings

In [1]:
# Libraries
import os
import sqlite3
import fnmatch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Function to close a sqlite db-connection
def check_conn(conn):
     try:
        conn.cursor()
        return True
     except Exception as ex:
        return False

# Get current working directory
print(os.getcwd())

/workspaces/data_analytics/Week_02


## Create sqlite data base

In [2]:
# Create data base
conn = sqlite3.connect('apartment_database.db') 
cursor = conn.cursor()

# Show dbs in the directory
flist = fnmatch.filter(os.listdir('.'), '*.db')
for i in flist:
    print(i)

supermarkets.db
apartment_database.db


## Create SQL-table in the database

In [4]:
cursor.execute('''CREATE TABLE IF NOT EXISTS apartments_table (OrderId VARCHAR(50),
                                                               Adress VARCHAR(200),
                                                               Rooms DECIMAL(8,2),
                                                               Area INT(8),
                                                               Price DECIMAL(8,2))''')
# Confirm changes to the table
conn.commit()

## Read data from file to data frame

In [5]:
df = pd.read_csv('supermarkets_data_prepared.csv', 
                  sep=',', 
                  encoding='utf-8')[['id', 'city', 'brand']]
print(df.shape)
df.head(5)

(3392, 3)


,id,city,brand
0,33126515,NaN,Spar
1,36726161,Uznach,Migros
2,39768209,Uznach,Coop
3,39947904,Zürich,Coop
4,48932835,Zürich,Migros


## Write data to the SQL-table in data base

In [6]:
df.to_sql(name = 'apartments_table',
          con = conn,
          index = False,
          if_exists = 'replace')

3392

## Query the SQL-table

In [7]:
# Query the SQL-table
cursor.execute('''SELECT *
               FROM supermarkets_table
               WHERE city =="Winterthur"''')

df = pd.DataFrame(cursor.fetchall(), 
                  columns=['Id','City','Brand'])    
df

,Id,City,Brand


## Additional SQL Queries

In [8]:
# SQL-Abfrage: Apartments mit ≥ 4 Zimmern und ≥ 100 m² Fläche
query = '''
SELECT *
FROM apartments_table
WHERE rooms >= 4.0 AND area >= 100
'''
result = cursor.execute(query).fetchall()
print("Apartments mit ≥ 4 Zimmern und ≥ 100 m² Fläche:")
for row in result:
    print(row)

# SQL-Abfrage: Durchschnittspreis pro Zimmergröße
query = '''
SELECT rooms, AVG(price) AS avg_price
FROM apartments_table
GROUP BY rooms
'''
result = cursor.execute(query).fetchall()
print("Durchschnittspreis pro Zimmergröße:")
for row in result:
    print(row)

# SQL-Abfrage: Durchschnittsfläche pro Zimmergröße
query = '''
SELECT rooms, AVG(area) AS avg_area
FROM apartments_table
GROUP BY rooms
'''
result = cursor.execute(query).fetchall()
print("Durchschnittsfläche pro Zimmergröße:")
for row in result:
    print(row)


OperationalError: no such column: rooms

## Plot histogramm of rental prices

In [ ]:
df.Price.plot.hist(grid=True, 
                   bins=20, 
                   rwidth=0.9,
                   color='#607c8e')
plt.title('Apartment price')
plt.xlabel('Price')
plt.ylabel('Frequency')
plt.grid(axis='y', alpha=0.75)

## Close db connection (if open)

In [ ]:
# Close db connection (if open)
try:
    if check_conn(conn):
        conn.close()
    else:
        pass
except:
    pass

# Status (True = open, False = closed)
print(check_conn(conn))

### Jupyter notebook --footer info-- (please always provide this at the end of each submitted notebook)

In [ ]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')